# PROCESS MAPPING V8 → V12

## Installer les librairies nécessaires

In [1]:
import pandas as pd
# Afficher le DataFrame sans limitation en largeur
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

import json
import re
import sqlglot
import pyodbc
import csv
import xml.etree.ElementTree as ET

## Exploiter l'EOM Sage V12

>Ce fichier utilise des données provenant de Sage X3 - V12
>- Utilisation de l'EOM V12 et notamment : 
>    - Table Principale
>    - Tables Liées
>    - Zones Parametrées

In [2]:
# Fichier EOM Sage V8-V12
eom_file = r".\data\Mapping_from_EOM_ZoneParamV12.xlsx"

In [3]:
from openpyxl import load_workbook

# Charger le fichier Excel et lire l'onglet "Mapping_V8V12" du fichier "Mapping_from_EOM_ZoneParamV12.xlsx" avec openpyxl
wb = load_workbook(filename=eom_file, data_only=True)
sheet = wb['Mapping_V8V12']

# Lire les données de la feuille et les convertir en dataframe
data = sheet.values
columns = next(data)[0:]
df_eom_raw = pd.DataFrame(data, columns=columns)

# Sélectionner les colonnes d'intérêt et les renommer
df_eom = df_eom_raw[['TableChamp_V8', 'TableChamp_V12', 'Désignation_MyReport']].rename(columns={
    'TableChamp_V8': 'tablechamp_v8',
    'TableChamp_V12': 'tablechamp_v12',
    'Désignation_MyReport': 'designation_champ_sortie'
})

# Ne selectionner que les lignes valides du mapping, celles dont la valeur est différente de 0
df_eom = df_eom[df_eom["tablechamp_v12"]!=0].reset_index(drop=True)

print(df_eom .info())
df_eom.head(18)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   tablechamp_v8             63 non-null     object
 1   tablechamp_v12            63 non-null     object
 2   designation_champ_sortie  63 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB
None


,tablechamp_v8,tablechamp_v12,designation_champ_sortie
0,MACHINEFL.MACNUM,XRMWMACPAR00.MACNUM,Wag_Code Machine
1,MACHINEFL.MACTYP,XRMWMACPAR00.MACTYP,Wag_Code Catégorie de wagon
2,MACHINES.SALFCY,XRMWMACPAR00.SALFCY,Wag_PC Origine
3,MACHINEFL.SERNUM,XRMWMACPAR00.SERNUM,N° de série
4,YTECSPE1.XAIO,XRMW1TSP01.XAIO,Wag_Aptitude à l'interopérabilité
5,YPARCWAG.XLOG,XRMWMACMIL01.XLOG,Wag_code retour PMA
6,YPARCWAG.XPMA,XRMWMACMIL01.XPMA,Wag_Parcours moyen annuel
7,YPARCWAG.YBPCUSR,XRMWCONMAC.BPCUSR,Wag_Code client utilisateur
8,YPARCWAG.YCONREV,XRMWCONMAC.CONREV,Wag_contrat avenant
9,YPARCWAG.YDATNEXTREV,WXRMWPRGDAT.DATENEXTREV,Wag_Prochaine opération périodique


In [4]:
df_eom['table_v8'] = df_eom['tablechamp_v8'].str.split('.').str[0]
df_eom['champ_v8'] = df_eom['tablechamp_v8'].str.split('.').str[1]
df_eom['table_v12'] = df_eom['tablechamp_v12'].str.split('.').str[0]
df_eom['champ_v12'] = df_eom['tablechamp_v12'].str.split('.').str[1]
df_eom = df_eom[['designation_champ_sortie','tablechamp_v8','table_v8','champ_v8','tablechamp_v12','table_v12','champ_v12']]

In [5]:
df_eom

,designation_champ_sortie,tablechamp_v8,table_v8,champ_v8,tablechamp_v12,table_v12,champ_v12
0,Wag_Code Machine,MACHINEFL.MACNUM,MACHINEFL,MACNUM,XRMWMACPAR00.MACNUM,XRMWMACPAR00,MACNUM
1,Wag_Code Catégorie de wagon,MACHINEFL.MACTYP,MACHINEFL,MACTYP,XRMWMACPAR00.MACTYP,XRMWMACPAR00,MACTYP
2,Wag_PC Origine,MACHINES.SALFCY,MACHINES,SALFCY,XRMWMACPAR00.SALFCY,XRMWMACPAR00,SALFCY
3,N° de série,MACHINEFL.SERNUM,MACHINEFL,SERNUM,XRMWMACPAR00.SERNUM,XRMWMACPAR00,SERNUM
4,Wag_Aptitude à l'interopérabilité,YTECSPE1.XAIO,YTECSPE1,XAIO,XRMW1TSP01.XAIO,XRMW1TSP01,XAIO
5,Wag_code retour PMA,YPARCWAG.XLOG,YPARCWAG,XLOG,XRMWMACMIL01.XLOG,XRMWMACMIL01,XLOG
6,Wag_Parcours moyen annuel,YPARCWAG.XPMA,YPARCWAG,XPMA,XRMWMACMIL01.XPMA,XRMWMACMIL01,XPMA
7,Wag_Code client utilisateur,YPARCWAG.YBPCUSR,YPARCWAG,YBPCUSR,XRMWCONMAC.BPCUSR,XRMWCONMAC,BPCUSR
8,Wag_contrat avenant,YPARCWAG.YCONREV,YPARCWAG,YCONREV,XRMWCONMAC.CONREV,XRMWCONMAC,CONREV
9,Wag_Prochaine opération périodique,YPARCWAG.YDATNEXTREV,YPARCWAG,YDATNEXTREV,WXRMWPRGDAT.DATENEXTREV,WXRMWPRGDAT,DATENEXTREV


In [6]:
print(f"Il y a {df_eom.shape[0]} champs de sortie de modèle à mapper")

Il y a 63 champs de sortie de modèle à mapper


## Exploiter la Requête SQL du modèle V8

### Nettoyer la Requête SQL du modèle V8 provenant de MyReport

In [7]:
import re
import sqlparse

# Chemins des fichiers
sql_v8_file_raw = r".\data\SQLQuery_ParcWagonV8_Raw.sql"
sql_v8_file = r".\data\SQLQuery_ParcWagonV8.sql"

# Formater la requête SQL
def format_sql_query(file_path):
    with open(file_path, 'r') as fichier:
        sql = fichier.read()
    formatted_sql = sqlparse.format(sql, reindent=True, keyword_case='upper')
    return formatted_sql

# Supprimer les alias dans la clause SELECT
def remove_aliases(sql_content):
    clause_select = re.search(r'SELECT.*?(?=FROM|WHERE|GROUP|HAVING|ORDER|LIMIT)', sql_content, re.DOTALL)
    if clause_select:
        clause_select_sans_alias = re.sub(r'AS [a-zA-Z0-9]+_\d+_\d+', '', clause_select.group())
        sql_content = sql_content.replace(clause_select.group(), clause_select_sans_alias)
    return sql_content

# Supprimer les espaces après les guillemets doubles dans la clause SELECT
def remove_spaces_after_quotes(sql_content):
    match = re.search(r'SELECT(.*?)FROM', sql_content, re.DOTALL | re.IGNORECASE | re.MULTILINE)
    if match:
        select_clause = match.group(1)
        select_clause = re.sub(r'"(\w+_\d+)"\s+', r'"\1"', select_clause)
        sql_content = re.sub(r'SELECT.*?FROM', f'SELECT{select_clause}\nFROM', sql_content, flags=re.DOTALL | re.IGNORECASE | re.MULTILINE)
    return sql_content

# Ajouter le mot clé AS entre X et Y hormis dans la clause SELECT
def add_as_keyword(sql_content):
    sql_content = re.sub(r'(ERMI01\.\w+)\s+(CVAR_DOSSIER__\w+)(?!(.*FROM.*SELECT.*))', r'\1 AS \2', sql_content, flags=re.DOTALL | re.IGNORECASE | re.MULTILINE)
    return sql_content

# Supprimer les doubles quotes
def remove_double_quotes(sql_content):
    sql_content = sql_content.replace('"', '')
    return sql_content

# Exécution des étapes
formatted_sql = format_sql_query(sql_v8_file_raw)
sql_content = remove_aliases(formatted_sql)
sql_content = remove_spaces_after_quotes(sql_content)
sql_content = add_as_keyword(sql_content)
sql_content = remove_double_quotes(sql_content)

# Enregistrement du résultat
with open(sql_v8_file, 'w') as fichier:
    fichier.write(sql_content)

### Analyser la Requête - Avec prise en compte des éléments parasites "Tableau des Charges"

In [8]:
sql_v8_file = r".\data\SQLQuery_ParcWagonV8.sql"

In [9]:
# Fichier de sortie (résultats) de la requête SQL v8
Result_SQLQueryV8= r".\data\Result_SQLQueryV8.csv"

In [10]:
# Elemennts de connexion à la Base de Données SQL de Production ; ERMI01
server_prod="frx3sql01.ms.ermi.systems\\ERSX3"
user_prod=None
password_prod=None
db_prod="ersx3"
schema_prod="ERMI01"
driver = "ODBC Driver 17 for SQL Server"

In [11]:
connection_string = f'DRIVER={driver};SERVER={server_prod};PORT=1433;DATABASE={db_prod};SCHEMA={schema_prod};Trusted_Connection=yes;&autocommit=true'
print(connection_string)

DRIVER=ODBC Driver 17 for SQL Server;SERVER=frx3sql01.ms.ermi.systems\ERSX3;PORT=1433;DATABASE=ersx3;SCHEMA=ERMI01;Trusted_Connection=yes;&autocommit=true


In [12]:
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [13]:
# Lire la requête SQL à partir du fichier
with open(sql_v8_file, 'r') as f:
    query = f.read()

# Exécuter la requête SQL
cursor.execute(query)

# Récupérer les résultats
results = cursor.fetchall()

In [14]:
# Ouvrir le fichier CSV en mode écriture
with open(Result_SQLQueryV8, 'w', newline='',encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)

    # Écrire les en-têtes de colonne
    writer.writerow([desc[0] for desc in cursor.description])

    # Écrire les lignes de résultats
    writer.writerows(results)

In [15]:
# Fermer curseur et connexion
cursor.close()
conn.close()

In [16]:
# Créer un dataframe des résultats de la requête SQL v8
# Ici nous avons en colonnes les champs de sortie du modèle
df_result_sqlv8 = pd.read_csv(Result_SQLQueryV8, encoding="utf-8", low_memory=False)

In [17]:
df_result_sqlv8.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104605 entries, 0 to 104604
Data columns (total 239 columns):
 #    Column         Dtype  
---   ------         -----  
 0    SERNUM_0       int64  
 1    MACNUM_0       int64  
 2    YSA01_0        int64  
 3    YTA03_0        object 
 4    MACTYP_0       object 
 5    YSA04_0        object 
 6    YSA09_0        object 
 7    YTA08_0        object 
 8    YSG04_0        object 
 9    YSG05_0        object 
 10   YSG06_0        object 
 11   YSG07_0        object 
 12   YSG08_0        object 
 13   YSG17_0        object 
 14   YTA02_0        object 
 15   YSG01_0        object 
 16   YSG02_0        object 
 17   YTA05_0        object 
 18   ZCATGEST_0     object 
 19   ZCATCOMM_0     object 
 20   YSG28_0        object 
 21   YSTATITM_0     object 
 22   YSA41_0        object 
 23   SALFCY_0       object 
 24   YSG33_0        object 
 25   YTL01_0        object 
 26   YTL05_0        object 
 27   ZRSRDFLG_0     int64  
 28   ZRSRDMAJ_0  

In [18]:
print(f"En prenant en compte 'Tableau des Charges' il y a ici {df_result_sqlv8.shape[1]} champs de sortie du modèle")
print('\n')
print(f"En prenant en compte 'Tableau des Charges' il y a ici {df_result_sqlv8.shape[0]} enregistrements (lignes) en sortie de modèle")

En prenant en compte 'Tableau des Charges' il y a ici 239 champs de sortie du modèle


En prenant en compte 'Tableau des Charges' il y a ici 104605 enregistrements (lignes) en sortie de modèle


### Analyser la Requête - Sans prise en compte des éléments parasites "Tableau des Charges"

Il faut faire abstraction (supprimer) la table "YTECSPE5" des résultats de l'analyse des tables et champs dans toutes les clauses de la requête SQL v8. 

In [19]:
import sqlglot

def get_tables_and_columns(query_file_path):
    tables = []
    
    # Lire la requête SQL depuis le fichier
    with open(query_file_path, 'r') as file:
        query = file.read()

    # Trouver toutes les tables et colonnes dans la requête
    table_column_pattern = re.compile(r'(\w+)\.(\w+)')
    matches = table_column_pattern.findall(query)

    for table, column in matches:
        tables.append({'table': table, 'champ': column})

    return tables

In [20]:
# Créer les DataFrames
tables = get_tables_and_columns(sql_v8_file)

# Créer le DataFrame pour les tables et colonnes
df_table = pd.DataFrame(tables)

# Trier le DataFrame selon les colonnes 'table' et 'champ'
df_table.sort_values(by=['table','champ'],inplace=True)

# Créer une copie pour df_table_unique
df_table_unique = df_table.copy(deep=True)

# Supprimer les lignes dont les valeurs contiennent "YTECSPE5" dans la colonne "table"
df_table = df_table[(~df_table['table'].str.contains('YTECSPE5')) & (~df_table['table'].str.contains(schema_prod))]

# Dédoublonner le DataFrame afin de ne voir qu'une occurrence de "table" - "champ"
df_table= df_table.drop_duplicates(subset=['table', 'champ']).reset_index(drop=True)

In [21]:
df_table_unique = df_table_unique[(df_table_unique["table"]==schema_prod) & (~df_table_unique['champ'].str.contains('YTECSPE5'))]
df_table_unique.rename(columns={"table":"env", "champ":"table"},inplace=True)

In [22]:
df_table_unique

,env,table
240,ERMI01,MACHINEFL
239,ERMI01,MACHINES
249,ERMI01,YMAINPLAN
246,ERMI01,YPARCWAG
243,ERMI01,YTECSPE1


In [23]:
print(f"Sans prendre en compte 'Tableau des Charges'\nil y a {df_table_unique.shape[0]} tables uniques impliquées\nSoit un minimum de {df_table_unique.shape[0]-1} champs pour les jointures")

Sans prendre en compte 'Tableau des Charges'
il y a 5 tables uniques impliquées
Soit un minimum de 4 champs pour les jointures


In [24]:
df_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   table   67 non-null     object
 1   champ   67 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [25]:
df_table[df_table["table"].str.contains('YTECSPE5')]

,table,champ


In [26]:
print(f"Sans prendre en compte 'Tableau des Charges'\nil y a {df_table.shape[0]} champs utilisés\n  En otant les {df_table_unique.shape[0]-1} champs pour les jointures\n  il y a {df_table.shape[0]-(df_table_unique.shape[0]-1)} champs de sortie dans le modèle")

Sans prendre en compte 'Tableau des Charges'
il y a 67 champs utilisés
  En otant les 4 champs pour les jointures
  il y a 63 champs de sortie dans le modèle


In [27]:
df_table.head(18)

,table,champ
0,CVAR_DOSSIER__MACHINEFL,MACNUM_0
1,CVAR_DOSSIER__MACHINEFL,MACTYP_0
2,CVAR_DOSSIER__MACHINEFL,SERNUM_0
3,CVAR_DOSSIER__MACHINEFL,YPLNMCE_0
4,CVAR_DOSSIER__MACHINEFL,YSA01_0
5,CVAR_DOSSIER__MACHINEFL,YSA04_0
6,CVAR_DOSSIER__MACHINEFL,YSA09_0
7,CVAR_DOSSIER__MACHINEFL,YSA41_0
8,CVAR_DOSSIER__MACHINEFL,YSG01_0
9,CVAR_DOSSIER__MACHINEFL,YSG02_0


In [28]:
# Création de la nouvelle colonne "new_c" en extrayant le groupe à droite du dernier underscore
df_table['table_v8'] = df_table['table'].str.extract('CVAR_DOSSIER__(.*)$')
df_table['champ_v8'] = df_table['champ'].str.removesuffix('_0')
df_table["tablechamp_v8"]=(df_table["table_v8"]+"."+df_table["champ_v8"])
df_table = df_table[["table","champ","table_v8","champ_v8","tablechamp_v8"]]

In [29]:
df_table

,table,champ,table_v8,champ_v8,tablechamp_v8
0,CVAR_DOSSIER__MACHINEFL,MACNUM_0,MACHINEFL,MACNUM,MACHINEFL.MACNUM
1,CVAR_DOSSIER__MACHINEFL,MACTYP_0,MACHINEFL,MACTYP,MACHINEFL.MACTYP
2,CVAR_DOSSIER__MACHINEFL,SERNUM_0,MACHINEFL,SERNUM,MACHINEFL.SERNUM
3,CVAR_DOSSIER__MACHINEFL,YPLNMCE_0,MACHINEFL,YPLNMCE,MACHINEFL.YPLNMCE
4,CVAR_DOSSIER__MACHINEFL,YSA01_0,MACHINEFL,YSA01,MACHINEFL.YSA01
5,CVAR_DOSSIER__MACHINEFL,YSA04_0,MACHINEFL,YSA04,MACHINEFL.YSA04
6,CVAR_DOSSIER__MACHINEFL,YSA09_0,MACHINEFL,YSA09,MACHINEFL.YSA09
7,CVAR_DOSSIER__MACHINEFL,YSA41_0,MACHINEFL,YSA41,MACHINEFL.YSA41
8,CVAR_DOSSIER__MACHINEFL,YSG01_0,MACHINEFL,YSG01,MACHINEFL.YSG01
9,CVAR_DOSSIER__MACHINEFL,YSG02_0,MACHINEFL,YSG02,MACHINEFL.YSG02


In [30]:
# Fusionner (merge) du dataframe provenant de l'EOM et du dataframe provenant de la requête SQL v8
df_table_m = pd.merge(
    left=df_table,
    right=df_eom,
    on="tablechamp_v8",
    how="outer",
    indicator=True
    )
df_table_m.drop(columns=["table","champ","table_v8_y","champ_v8_y"],inplace=True)
df_table_m = df_table_m[["designation_champ_sortie","tablechamp_v8","table_v8_x","champ_v8_x","tablechamp_v12","table_v12","champ_v12","_merge"]]
df_table_m.rename(columns={"table_v8_x":"table_v8", "champ_v8_x":"champ_v8"},inplace=True)
df_table_m

,designation_champ_sortie,tablechamp_v8,table_v8,champ_v8,tablechamp_v12,table_v12,champ_v12,_merge
0,Wag_Code Machine,MACHINEFL.MACNUM,MACHINEFL,MACNUM,XRMWMACPAR00.MACNUM,XRMWMACPAR00,MACNUM,both
1,Wag_Code Catégorie de wagon,MACHINEFL.MACTYP,MACHINEFL,MACTYP,XRMWMACPAR00.MACTYP,XRMWMACPAR00,MACTYP,both
2,N° de série,MACHINEFL.SERNUM,MACHINEFL,SERNUM,XRMWMACPAR00.SERNUM,XRMWMACPAR00,SERNUM,both
3,N° Plan maintenance,MACHINEFL.YPLNMCE,MACHINEFL,YPLNMCE,XRMWMPL.YMPNUM,XRMWMPL,YMPNUM,both
4,Wag_N° Immatriculation,MACHINEFL.YSA01,MACHINEFL,YSA01,XRMWMACIMM00.YHI00,XRMWMACIMM00,YHI00,both
5,Wag_Pool mat (gérant),MACHINEFL.YSA04,MACHINEFL,YSA04,XRMWMACPAR01.YSA04,XRMWMACPAR01,YSA04,both
6,Wag_Etat du matériel,MACHINEFL.YSA09,MACHINEFL,YSA09,XRMWMACPAR01.YSA09,XRMWMACPAR01,YSA09,both
7,Wag_Code Pool Essieu,MACHINEFL.YSA41,MACHINEFL,YSA41,XRMWMACPAR01.YSA41,XRMWMACPAR01,YSA41,both
8,Wag_Date d'entrée sur parc,MACHINEFL.YSG01,MACHINEFL,YSG01,XRMWMACPAR01.YSG01,XRMWMACPAR01,YSG01,both
9,Wag_Date désimmatriculation,MACHINEFL.YSG02,MACHINEFL,YSG02,XRMWMACPAR01.YSG02,XRMWMACPAR01,YSG02,both


In [31]:
df_champ_absent_v12 = df_table_m[df_table_m["_merge"]=="left_only"]
df_champ_absent_v12

,designation_champ_sortie,tablechamp_v8,table_v8,champ_v8,tablechamp_v12,table_v12,champ_v12,_merge
26,NaN,MACHINES.MACNUM,MACHINES,MACNUM,NaN,NaN,NaN,left_only
29,NaN,YMAINPLAN.YMPNUM,YMAINPLAN,YMPNUM,NaN,NaN,NaN,left_only
30,NaN,YPARCWAG.SERNUM,YPARCWAG,SERNUM,NaN,NaN,NaN,left_only
38,NaN,YTECSPE1.SERNUM,YTECSPE1,SERNUM,NaN,NaN,NaN,left_only


## Etat consolidé pour "Requête SQL du modèle V8 - EOM Sage V12"

In [32]:
df_champ_v8v12 = df_table_m[df_table_m["_merge"]=="both"].reset_index(drop=True)
df_champ_v8v12

,designation_champ_sortie,tablechamp_v8,table_v8,champ_v8,tablechamp_v12,table_v12,champ_v12,_merge
0,Wag_Code Machine,MACHINEFL.MACNUM,MACHINEFL,MACNUM,XRMWMACPAR00.MACNUM,XRMWMACPAR00,MACNUM,both
1,Wag_Code Catégorie de wagon,MACHINEFL.MACTYP,MACHINEFL,MACTYP,XRMWMACPAR00.MACTYP,XRMWMACPAR00,MACTYP,both
2,N° de série,MACHINEFL.SERNUM,MACHINEFL,SERNUM,XRMWMACPAR00.SERNUM,XRMWMACPAR00,SERNUM,both
3,N° Plan maintenance,MACHINEFL.YPLNMCE,MACHINEFL,YPLNMCE,XRMWMPL.YMPNUM,XRMWMPL,YMPNUM,both
4,Wag_N° Immatriculation,MACHINEFL.YSA01,MACHINEFL,YSA01,XRMWMACIMM00.YHI00,XRMWMACIMM00,YHI00,both
5,Wag_Pool mat (gérant),MACHINEFL.YSA04,MACHINEFL,YSA04,XRMWMACPAR01.YSA04,XRMWMACPAR01,YSA04,both
6,Wag_Etat du matériel,MACHINEFL.YSA09,MACHINEFL,YSA09,XRMWMACPAR01.YSA09,XRMWMACPAR01,YSA09,both
7,Wag_Code Pool Essieu,MACHINEFL.YSA41,MACHINEFL,YSA41,XRMWMACPAR01.YSA41,XRMWMACPAR01,YSA41,both
8,Wag_Date d'entrée sur parc,MACHINEFL.YSG01,MACHINEFL,YSG01,XRMWMACPAR01.YSG01,XRMWMACPAR01,YSG01,both
9,Wag_Date désimmatriculation,MACHINEFL.YSG02,MACHINEFL,YSG02,XRMWMACPAR01.YSG02,XRMWMACPAR01,YSG02,both


## Exploiter le fichier MDLX du modèle V8

### Créer un JSON du Fichier du modèle V8 - Avec prise en compte des éléments parasites "Tableau des Charges"

In [33]:
mdlx_v8_file = r".\data\ParcWagonV8.mdlx" # fichier d'entrée
json_v8_file = r".\data\model_v8.json" # fichier de sortie

In [34]:
import xml.etree.ElementTree as ET
import json

# Charger le fichier MDLX avec encoding UTF-8
tree = ET.parse(mdlx_v8_file, parser=ET.XMLParser(encoding='utf-8'))
root = tree.getroot()

# Fonction récursive pour parser l'arbre XML
def parse_node(node):
    data = {}
    for child in node:
        if len(child) > 0:
            if child.tag in data:
                if not isinstance(data[child.tag], list):
                    data[child.tag] = [data[child.tag]]
                data[child.tag].append(parse_node(child))
            else:
                data[child.tag] = parse_node(child)
        else:
            if child.tag in data:
                if not isinstance(data[child.tag], list):
                    data[child.tag] = [data[child.tag]]
                data[child.tag].append(child.text)
            else:
                data[child.tag] = child.text
        # Ajouter les attributs
        for attr in child.attrib:
            if f"{child.tag}_{attr}" in data:
                if not isinstance(data[f"{child.tag}_{attr}"], list):
                    data[f"{child.tag}_{attr}"] = [data[f"{child.tag}_{attr}"]]
                data[f"{child.tag}_{attr}"].append(child.attrib[attr])
            else:
                data[f"{child.tag}_{attr}"] = child.attrib[attr]
    # Ajouter les attributs du nœud actuel
    for attr in node.attrib:
        if f"{node.tag}_{attr}" in data:
            if not isinstance(data[f"{node.tag}_{attr}"], list):
                data[f"{node.tag}_{attr}"] = [data[f"{node.tag}_{attr}"]]
            data[f"{node.tag}_{attr}"].append(node.attrib[attr])
        else:
            data[f"{node.tag}_{attr}"] = node.attrib[attr]
    return data

# Parser l'arbre XML
data = parse_node(root)

# Créer le fichier JSON avec encoding UTF-8
with open(json_v8_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

Lors de l'analyse du fichier XML, nous avons ajouté le paramètre parser à ET.parse() et spécifié une instance XMLParser avec encoding='utf-8'. 
- Cela indique à ElementTree qu'il doit utiliser l'encodage UTF-8 lors de la lecture du fichier XML.

Lors de l'écriture du fichier JSON, nous avons ajouté le paramètre ensure_ascii=False à json.dump(). 
- Cela indique au sérialiseur JSON qu'il doit préserver les caractères non ASCII (comme les accents) au lieu de les escamoter.

## Exploiter le fichier MDLX du modèle V12

### Créer un JSON du Fichier du modèle V12 - Sans prise en compte des éléments parasites "Tableau des Charges"

In [35]:
mdlx_v12_file = r".\data\ParcWagonV12.mdlx" # fichier d'entrée
json_v12_file = r".\data\model_v12.json" # fichier de sortie

In [36]:
import xml.etree.ElementTree as ET
import json

# Charger le fichier MDLX avec encoding UTF-8
tree = ET.parse(mdlx_v12_file, parser=ET.XMLParser(encoding='utf-8'))
root = tree.getroot()

# Fonction récursive pour parser l'arbre XML
def parse_node(node):
    data = {}
    for child in node:
        if len(child) > 0:
            if child.tag in data:
                if not isinstance(data[child.tag], list):
                    data[child.tag] = [data[child.tag]]
                data[child.tag].append(parse_node(child))
            else:
                data[child.tag] = parse_node(child)
        else:
            if child.tag in data:
                if not isinstance(data[child.tag], list):
                    data[child.tag] = [data[child.tag]]
                data[child.tag].append(child.text)
            else:
                data[child.tag] = child.text
        # Ajouter les attributs
        for attr in child.attrib:
            if f"{child.tag}_{attr}" in data:
                if not isinstance(data[f"{child.tag}_{attr}"], list):
                    data[f"{child.tag}_{attr}"] = [data[f"{child.tag}_{attr}"]]
                data[f"{child.tag}_{attr}"].append(child.attrib[attr])
            else:
                data[f"{child.tag}_{attr}"] = child.attrib[attr]
    # Ajouter les attributs du nœud actuel
    for attr in node.attrib:
        if f"{node.tag}_{attr}" in data:
            if not isinstance(data[f"{node.tag}_{attr}"], list):
                data[f"{node.tag}_{attr}"] = [data[f"{node.tag}_{attr}"]]
            data[f"{node.tag}_{attr}"].append(node.attrib[attr])
        else:
            data[f"{node.tag}_{attr}"] = node.attrib[attr]
    return data

# Parser l'arbre XML
data = parse_node(root)

# Créer le fichier JSON avec encoding UTF-8
with open(json_v12_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)